In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from openai import OpenAI
import pandas as pd

# Load models
tokenizer1 = AutoTokenizer.from_pretrained("EleutherAI/pythia-1b")
model1 = AutoModelForSeq2SeqLM.from_pretrained("EleutherAI/pythia-1b")

tokenizer2 = AutoTokenizer.from_pretrained("/models/pythia-1b-fine-tuned/")
model2 = AutoModelForSeq2SeqLM.from_pretrained("/models/pythia-1b-fine-tuned/checkpoint-3600")

# Initialize OpenAI API for text-davinci-003
openai.api_key = 'your-api-key'
text_davinci_003 = openai.Completion.create(engine="text-davinci-003")

# Prepare test questions
test_questions = [...]  # replace with your list of questions

# Save predictions
predictions1 = []
predictions2 = []
for question in test_questions:
    input_ids = tokenizer1.encode(question, return_tensors='pt')
    output_ids = model1.generate(input_ids)
    prediction1 = tokenizer1.decode(output_ids[0], skip_special_tokens=True)
    predictions1.append(prediction1)

    input_ids = tokenizer2.encode(question, return_tensors='pt')
    output_ids = model2.generate(input_ids)
    prediction2 = tokenizer2.decode(output_ids[0], skip_special_tokens=True)
    predictions2.append(prediction2)

# Compare predictions
judge_response = []
for pred1, pred2 in zip(predictions1, predictions2):
    prompt = {
        "prompt_id": 1,
        "system_prompt": "You are a helpful and precise assistant for checking the quality of the answer.",
        "prompt_template": f"[Question]\n{question}\n\n[The Start of Assistant 1's Answer]\n{pred1}\n\n[The End of Assistant 1's Answer]\n\n[The Start of Assistant 2's Answer]\n{pred2}\n\n[The End of Assistant 2's Answer]\n\n[System]\n{prompt}\n\n",
        "defaults": {
            "prompt": "We would like to request your feedback on the performance of two AI assistants in response to the user question displayed above.\nPlease rate the helpfulness, relevance, accuracy, level of details of their responses. Each assistant receives an overall score on a scale of 1 to 10, where a higher score indicates better overall performance.\nPlease first output a single line containing only two values indicating the scores for Assistant 1 and 2, respectively. The two scores are separated by a space. In the subsequent line, please provide a comprehensive explanation of your evaluation, avoiding any potential bias and ensuring that the order in which the responses were presented does not affect your judgment."
        },
        "description": "Prompt for general questions",
        "category": "general"
    }
    response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  # The name of the OpenAI chatbot model to use
            messages=prompt,   # The conversation history up to this point, as a list of dictionaries
            max_tokens=200,        # The maximum number of tokens (words or subwords) in the generated response
            stop=None,              # The stopping sequence for the generated response, if any (not used here)
            temperature=0.7,        # The "creativity" of the generated response (higher temperature = more creative)
        )
    judge_response.append(response)  # Assuming the first score is for the first model

df = pd.DataFrame(data = [predictions1, predictions2, judge_response], columns = ['Pythia-1b', 'Pythia-1b fine-tuned', 'text-davinci-003 response', 'winner'])

# Extract the winner from the 'text-davinci-003 response' column
df['winner'] = df['text-davinci-003 response'].apply(lambda x: x.choices[0].text.strip().split()[0])

# Count the number of times each model won
model1_wins = (df['winner'] == '1').sum()
model2_wins = (df['winner'] == '2').sum()

# Print results
print(f"Model 1 (Pythia-1b) won {model1_wins} times.")
print(f"Model 2 (Pythia-1b fine-tuned) won {model2_wins} times.")